# Maneuvres d'un satellite

On décrit le mouvement d'un objet de masse $m$, dans un référentiel galiléen, avec des forces de frottement négligeables.

À l'aide de la deuxième loi de Newton ou le Principe Fondamental de la dynamique (PFD) on a:

$$
\sum_i \vec{F_i} = m . \vec{a} 
$$

## Ellipse

Le mouvement d'un satellite porsuivre une trajectoire donnée par les sections coniques. On va s'intéresser pour des cas où la trajectoire donne comme résultat une ellipse.

Soient $(d)$ une droite verticale qu'on appel `directrice`, $F$ un point qui n'appartient pas à $(di)$, qu'on appellarait `foyer`, $M$ un `point` sur l'ellipse, et $d$ comme la `distance` de $M$ jusqu'à $F$.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Ellipse_foyer_directrice2.svg/660px-Ellipse_foyer_directrice2.svg.png" style="display:block;margin:auto;width:400px" />

À partir de ce raisonnement on peut trouver une rélacion qu'on appel l'`excentricité` $e$.

$$
e = \frac{d(M,F)}{d(M,(d))}
$$

Dans l'ellipse on peut définir les grandeurs présents:

- `a` demi grand axe
- `b` demi petit axe
- `c` distance entre le centre el le foyer
- `h` distance avec la directrice et le foyer
- `e` excentricité $\in [0,1]$ 
- `p` demi lactus rectum

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/EllipseVal.svg/880px-EllipseVal.svg.png" style="display:block;margin:auto;width:400px" >

Dans le plane cartesienne, l'équation de l'ellipse est donné par: 

$$
\left( \frac{x}{a} \right)^2 + \left( \frac{y}{b} \right)^2 = 1
$$

Avec $a \gt b \gt 0 $.

La distance du centre jusqu'à l'une de foyers: 

$$
c = \sqrt{a^2 - b^2} 
$$

L'excentricité $e$:

$$
e = \frac{c}{a} = \frac{ \sqrt{a^2 - b^2} }{a} =  \sqrt{ 1 - \left(\frac{b}{a}\right)^2}
$$